# Data Preparation
## Student Names: Aftab Alam, Harpreet Singh, Srijani Basak, Srinivas Murri, Vineeta Singla
## Course: IMT 575

In [101]:
import pandas as pd
import numpy as np
import re
import sklearn
import random
from random import choices
from random import sample 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.corpus import words
nltk.download('words')
import datetime

transtable = str.maketrans('', '', string.punctuation)
def removePunc(x):
   return x.translate(transtable)



### Collect Benign Domains from Google Search using Google API

In [2]:
benignDF = pd.read_csv("./newCleanDomains.csv",header=0)
benignDF = benignDF[benignDF['domain_type'] == 'benign']
benignDF.drop_duplicates(keep = 'first', inplace = True) 
benignDF['domain_type'] = [element.lower() for element in benignDF['domain_type']] ; benignDF['domain_type']
benignDF['domain_type'].value_counts()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


benign    637526
Name: domain_type, dtype: int64

In [3]:
ipRegex = "^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$"

def isIpAddress(domain):
    try:
        if(re.match(ipRegex, domain)):
            return True
    except:
        return False
    return False
    
def getTlds(domain):
    try:
        if(not isIpAddress(domain)):
            index = domain.rindex('.')
            val = domain[index:]
            if(len(val)>1):
                return val
    except:
        domain = ""    

def getSlds(domain):
    try:
        if(not isIpAddress(domain)):
            domain = domain.replace('www.', '')
            segments = domain.split(".")
            count = len(segments)
            if(count > 3):
                return "."+segments[count-2]
    except:
        domain=""

def getPopularShortTld(domain):
     if(not isIpAddress(domain)):
        try:
            index = domain.rindex('.')
            index2 = domain[0:index].rindex('.')
            val = domain[index2:]
            if(len(val)<=7):
                return val
        except:
            domain = ""

In [4]:
sponseredTldSet = set(['.aero','.asia','.cat','.coop','.edu','.gov','.int','.jobs','.mil','.museum','.post','.tel','.travel','.xxx'])
countryTldSet = set(['.ac','.ad','.ae','.af','.ag','.ai','.al','.am','.ao','.aq','.ar','.as','.at','.au','.aw','.ax','.az','.ba','.bb','.bd','.be','.bf','.bg','.bh','.bi','.bj','.bm','.bn','.bo','.br','.bs','.bt','.bw','.by','.bz','.ca','.cc','.cd','.cf','.cg','.ch','.ci','.ck','.cl','.cm','.cn','.co','.cr','.cu','.cv','.cw','.cx','.cy','.cz','.de','.dj','.dk','.dm','.do','.dz','.ec','.ee','.eg','.er','.es','.et','.eu','.fi','.fj','.fk','.fm','.fo','.fr','.ga','.gd','.ge','.gf','.gg','.gh','.gi','.gl','.gm','.gn','.gp','.gq','.gr','.gs','.gt','.gu','.gw','.gy','.hk','.hm','.hn','.hr','.ht','.hu','.id','.ie','.il','.im','.in','.io','.iq','.ir','.is','.it','.je','.jm','.jo','.jp','.ke','.kg','.kh','.ki','.km','.kn','.kp','.kr','.kw','.ky','.kz','.la','.lb','.lc','.li','.lk','.lr','.ls','.lt','.lu','.lv','.ly','.ma','.mc','.md','.me','.mg','.mh','.mk','.ml','.mm','.mn','.mo','.mp','.mq','.mr','.ms','.mt','.mu','.mv','.mw','.mx','.my','.mz','.na','.nc','.ne','.nf','.ng','.ni','.nl','.no','.np','.nr','.nu','.nz','.om','.pa','.pe','.pf','.pg','.ph','.pk','.pl','.pm','.pn','.pr','.ps','.pt','.pw','.py','.qa','.re','.ro','.rs','.ru','.rw','.sa','.sb','.sc','.sd','.se','.sg','.sh','.si','.sk','.sl','.sm','.sn','.so','.sr','.ss','.st','.su','.sv','.sx','.sy','.sz','.tc','.td','.tf','.tg','.th','.tj','.tk','.tl','.tm','.tn','.to','.tr','.tt','.tv','.tw','.tz','.ua','.ug','.uk','.us','.uy','.uz','.va','.vc','.ve','.vg','.vi','.vn','.vu','.wf','.ws','.ye','.yt','.za','.zm','.zw'])
randomTlds = ['.ba','.kn','.love','.tp','.social','.is','.to','.co.ug','.org.np','.dhl','.jcb','.pa','.mh','.tips','.vip','.pub','.om','.br','.gratis','.ink','.host','.barclaycard','.law','.io','.jo','.tel','.sharp','.equipment','.direct','.pk','.tr','.cx','.community','.auction','.tc','.ls','.ac','.us','.gov.az','.gt','.tn','.cc','.ph','.co.za','.mi','.sm','.fk','.com.tw','.tv','.ca','.apple','.bh','.moi','.mg','.training','.ax','.jp','.bank','.org','.buzz','.es','.lk','.sz','.as','.id','.in','.tm','.eu','.app','.km','.gb','.media','.house','.services','.tz','.sg','.football','.exchange','.fg','.hm','.nyc','.il.us','.fun','.domains','.sv','.cl','.report','.ac.uk','.in.ua','.network','.td','.rent','.ms','.ci','.moscow','.uk','.ad','.ec','.agency','.news','.onl','.group','.com.vn','.ge','.biz','.lt','.zone','.movie','.best','.fo','.software','.gg','.an','.ua','.edu.in','.club','.eus','.me','.jobs','.education','.org.sg','.bot','.blog','.solutions','.co.zw','.cr','.nl','.au','.store','.yt','.sc','.hn','.win','.com.au','.cloud','.fox','.gs','.gp','.co','.py','.discount','.trade','.pr','.mt','.nj.us','.sy','.london','.usc.edu','.mobi','.co.jp','.hk','.lc','.fm','.garden','.expert','.bar','.so','.cv','.shop','.gov.np','.inc','.mz','.pink','.film','.systems','.fitness','.be','.bio','.help','.vu','.bd','.global','.go','.beer','.sj','.codes','.ve','.ae','.vn','.vi','.tk','.ml','.cy','.mc','.ro','.rs','.cg','.coop','.ck','.my','.gm','.download','.stream','.bradesco','.clinic','.link','.nf','.by','.ai','.komatsu','.sh','.weir','.com .edu','.bt','.website','.bj','.tech','.conery.io','.chat','.gal','.eg','.fans','.toys','.contractors','.hr','.tf','.gh','.one','.frl','.tools','.center','.se','.ngo','.im','.nz','.reviews','.do','.tattoo','.uvu.edu','.city','.express','.ug','.gl','.ooo','.wang','.et','.dj','.earth','.pet','.build','.sd','.wiki','.li','.email','.na','.pg','.co.ao','.academy','.cern','.ee','.ru','.net.ua','.delivery','.bf','.box','.rocks','.xyz','.zw','.school','.market','.guru','.af.mil','.or','.review','.la','.red','.nc','.ch','.nr','.asia','.miami','.az','.hsbc.com','.mk','.mil','.site','.llc','.bg','.bs','.studio','.mp','.marketing','.sl','.com.my','.icu','.bw','.lb','.kh','.fit','.de','.ky','.mv','.country','.af','.gob.gt','.edu:8080','.dz','.gi','.bible','.ki','.bo','.si.edu','.green','.at','.vg','.lu','.no','.it','.com.hk','.uy','.bz','.technology','.tube','.ne','.web.app','.blue','.vet','.video','.cat','.sk','.estate','.care','.ed','.aw','.bi','.bv','.game','.cards','.gq','.edu.co','.tt','.edu.az','.mq','.pl.ua','.sx','.kg','.ltd','.sex','.gf','.golf','.biz.ua','.xn','.work','.pw','.hu','.gov','.info','.dk','.mm','.sb','.ly','.kw','.international','.bnpparibas','.desi','.md','.sa','.life','.am','.mn','.sony','.bm','.art','.wtf','.ps','.pm','.world','.audio','.today','.je','.quebec','.dm','.capital','.il','.science','.th','.gn','.epa.gov','.durban','.edu','.lawyer','.za','.nu','.do.am','.company','.mo','.co.in','.gd','.com.ua','.tl','.cm','.edu.cn','.pt','.party','.ng','.ie','.ao','.ma','.click','.dev','.kim','.style','.su','.support','.gov.ir','.kz','.ws','.name','.lat','.menu','.fi','.pro','.ninja','.qa','.plus','.new','.com','.ag','.cd','.ist','.boutique','.bid','.al','.tj','.tw','.bnl.gov','.top','.gle','.aws','.ca.gov','.um','.gov.cn','.cz','.cool','.wf','.er','.show','.va','.vc','.live','.ke','.google','.bn','.aero','.games','.aq','.lr','.design','.zm','.casa','.pl','.int','.porn','.ht','.cash','.date','.kr','.bb','.engineering','.com.sa','.press','.sr','.yu','.ar','.mu','.iq','.cn','.energy','.cu','.pe','.biz .org .in','.realtor','.money','.fj','.gu','.ir','.digital','.pf','.kpmg','.farm','.bike','.gr','.church','.guide','.uz','.rw','.st','.gw','.cheap','.gifts','.xxx','.credit','.ga','.cam','.fund','.scot','.sale','.pn','.mw','.si','.cf','.online','.sn','.lv','.mx','.travel','.ni','.careers','.jm','.monster','.mr','.np','.re','.ye','.parts','.co.uk','.fishing','.net','.space','.run','.edu.my','.moe','.gy','.health','.co.kr','.com.br','.com.ng','.tg','.fr']


In [5]:
tldSet = set(benignDF['domain'].apply(getTlds))
sldSet = set(benignDF['domain'].apply(getSlds))
popularShortTldSet = set(benignDF['domain'].apply(getPopularShortTld))

### Function definition for DGA 

In [53]:
sw = stopwords.words('english')   
word_list = words.words()

def generate_random_domain(seed,tlds):
    try:
        domain = ""
        allChars = 'abcdefghijklmnopqrstuvwxyz0123456789' 
        onlyAlphabets = 'abcdefghijklmnopqrstuvwxyz'
        n = seed + random.randint(5, 9) 
        if(n%2 == 0):
            domain = domain.join(random.choices(allChars, k=n))
        else:
            domain = domain.join(random.choices(onlyAlphabets, k=n))

        if(n%3 == 0 or n%7 == 0):
            domain = 'www.'+domain
        if(n%5 == 0 and len(domain)>10):
            index = random.randint(0, 5)  + 5
            domain = domain[:index] + '-' + domain[index + 1:]

        val = (choices(tlds, k=1)[0])
        domain = domain + val
        if domain:
            return domain
    except:
        domiain = ""

def generate_random_domain_addwords(tlds):
    try:
        domain = ""
        addWord = removePunc(random.choices(sw, k=1)[0])
        onlyAlphabets = 'abcdefghijklmnopqrstuvwxyz'

        n = random.randint(5, 10) 

        domain = domain.join(random.choices(onlyAlphabets, k=n))

        if(n%3 == 0 or n%7 == 0):
            domain = 'www.'+ addWord + domain
        if(n%5 == 0 and len(domain)>10):
            index = random.randint(0, 5)  + 5
            domain = domain[:index] + addWord + domain[index + 1:]

        domain = domain + (choices(tlds, k=1)[0])
        if domain:
            return domain
    except:
        domain = ""

def generate_random_domain_current_time(tlds):
    try:
        domain = ""
        addWord = removePunc(random.choices(word_list, k=1)[0])
        seed = datetime.datetime.now().microsecond
        charSet = 'abcdefghijklmnopqrstuvwxyz' + 'addWord' 
        if(seed%2 == 0):
            charSet = charSet + str(seed)

        n = random.randint(6,10) 
        domain = domain.join(random.choices(charSet, k=n))

        if(n%7 == 0 ):
            domain = 'www.'+ addWord + domain
        if(n%3 == 0 and len(domain)>10):
            index = random.randint(0, 5)  + 5
            domain = domain[:index] + '-' + addWord + domain[index + 1:]

        domain = domain + (choices(tlds, k=1)[0])
        if domain:
            return domain
    except:
        domain = ""

def generate_random_domain_shiftbits(tlds):
    try:
        domain = ""
        seed = datetime.datetime.now().microsecond
        charSet = 'abcdefghijklmnopqrstuvwxyz' 

        extra = ''
        for i in range(0,random.randint(5,12) ,1):
            day = random.randint(1, 30)
            extra = ((day ^ (day << random.randint(1, 17))) >> random.randint(1, 17)) ^ ((day & 0xFFFFFFFE) << random.randint(1, 17))

        domain = domain.join(random.choices(charSet, k=random.randint(4, 11))) + chr(((extra) % 25) + 97)

        if(seed%7 == 0 ):
            domain = 'www.'+ domain

        domain = domain + (choices(tlds, k=1)[0])
        if domain:
            return domain
    except:
        domain = "" 

In [105]:
def generate_dga_domains(count):
    dgaDomainList1 = [generate_random_domain(random.randint(0, 9),list(popularShortTldSet)) for i in range(1,int(count/4))] ; dgaDomainList1
    dgaDomainList2 = [generate_random_domain(random.randint(0, 9),randomTlds) for i in range(1,int(count/4))] ; dgaDomainList2
    dgaDomainList3 = [generate_random_domain_addwords(list(countryTldSet)) for i in range(1,int(count/4))] ; dgaDomainList3
    dgaDomainList4 = [generate_random_domain_current_time(list(tldSet)) for i in range(1,int(count/8))] ; dgaDomainList4  
    dgaDomainList5 = [generate_random_domain_shiftbits(list(popularShortTldSet)) for i in range(1,int(count/8))] ; dgaDomainList5
    dgaDomainList6 = [generate_random_domain_shiftbits(list(sponseredTldSet)) for i in range(1,int(count/64))] ; dgaDomainList6

    dgaDomainList = dgaDomainList1 + dgaDomainList2 + dgaDomainList3 + dgaDomainList4 + dgaDomainList5 + dgaDomainList6
    dgaDomainSet = set(dgaDomainList)
    return set(sample(list(dgaDomainSet),count))


In [106]:
dgaDomainSet = generate_dga_domains(637526)

# Label the generated domains as 'dga' and convert into a dataframe        
dgaDF = pd.DataFrame({'domain': domain, 'domain_type':'dga'} for domain in dgaDomainSet)
dgaDF.drop_duplicates(keep = 'first', inplace = True) 
dgaDF['domain_type'].value_counts()

dga    637526
Name: domain_type, dtype: int64

### Merge the Benign and DGA domains and write to a csv

In [107]:
#Merge the benign and dga dataframmes
allDomains  = benignDF.append(dgaDF)

#Shuffle the combined dataframe
allDomains = sklearn.utils.shuffle(allDomains)

#Write the results to a csv
allDomains.to_csv('allDomains.csv',index=False,header=True)

### Read the generated csv and print a sample subset

In [59]:
read_domains = pd.read_csv("allDomains.csv")
read_domains.sample(15)

,domain,domain_type
326344,acroofers.com,benign
139390,lob8yq0z.ac.id,dga
89047,cndzys.com,benign
915818,telcopl.com,benign
945005,www.tdsiu-xeckcwrvs.social,dga
862438,www.918xfp1x3pfnys.green,dga
216065,www.rmoxvnk.co.ug,dga
877452,pcdyj.org.gh,dga
182503,eopdroeld.bw,dga
262090,molodeg33.ru,benign


In [60]:
read_domains['domain_type'].value_counts()

benign    637526
dga       633080
Name: domain_type, dtype: int64

In [61]:
633080/(637526+633080)

0.49825044112809164